# The MNIST dataset

In [8]:
import mxnet as mx
from mxnet import nd,autograd
from mxnet import gluon
import numpy as np

ctx = mx.cpu()

#data shape
batch_size = 64
num_inputs = 784
num_outputs = 10

# data transform
def transform(data,label):
    return data.astype(np.float32)/255,label.astype(np.float32)
    
#dataset
train_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=True,transform=transform),batch_size,shuffle=True)
test_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=False,transform=transform),batch_size,shuffle=False)

# Define the model 

In [9]:
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Dense(num_outputs))

# Parameter Initialization 

In [10]:
net.collect_params().initialize(mx.init.Normal(sigma=1.),ctx=ctx)

# Softmax Cross Entropy Loss 

In [11]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

# Optimizer 

In [12]:
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate': 0.1})

# Evaluation Metric 

In [13]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(ctx).reshape((-1,784))
        label = label.as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

evaluate_accuracy(test_data, net)

0.058999999999999997